In [ ]:
import random
import numpy as np
import matplotlib as plt
import json
import glob
import torch
import re

np.random.seed(42)
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/cs6471proj2/amazon

Mounted at /content/drive
/content/drive/MyDrive/cs6471proj2/amazon


# Data Generation for Bag of Words

In [ ]:
import re
import json

def normalize_sentence(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

def normalize_sentence_4glove(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?']+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

In [ ]:
done = ["reviews_Cell_Phones_and_Accessories_5", "reviews_Toys_and_Games_5", 
        "reviews_Office_Products_5", "reviews_Pet_Supplies_5", 'reviews_Tools_and_Home_Improvement_5',
        "reviews_Automotive_5", "reviews_Digital_Music_5", "reviews_Musical_Instruments_5", 
        "reviews_Clothing_Shoes_and_Jewelry_5", "reviews_Amazon_Instant_Video_5"]

In [ ]:
country_worth2lookat = {'china':['china', 'ch'], 'us':['usa', 'us', 'america', 'united states', 'the states'], 
                      'uk':['uk', 'england', 'britain', 'ireland', 'scotland'], 'japan':['japan'], 'mexico':['mexico'], 'germany': ['germany'], 
                      'canada':['canada'], 'india':['india'], 'australia':['australia'], 'taiwan':['taiwan'], 'vietnam':['vietnam'], 
                      'thailand':['thailand'], 'korea':['korea'], 'iraq':['iraq'], 'russia':['russia']} #15


nationality_worth2lookat = {'american': ['american'], 'english': ['english'], 'german':['german'], 
                          'chinese':['chinese'], 'british':['british', 'scottish', 'irish'], 
                          'spanish':['spanish'], 'french':['french'], 'japanese':['japanese'], 
                          'european':['european'], 'australian':['australian'], 
                          'african':['african'], 'asian':['asian'], 'italian':['italian'], 
                          'canadian':['canadian'],  'indian':['indian'], 
                          'russian':['russian'], 'mexican':['mexican']} # 17

# build word2label
word2label_country = {}
word2label_nationality = {}
for key, item in country_worth2lookat.items():
    for word in item:
        word2label_country[word] = key
for key, item in nationality_worth2lookat.items():
    for word in item:
        word2label_nationality[word] = key

# For country, review_TFIDF

## make data
'''
https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/
https://stackabuse.com/implementing-word2vec-with-gensim-library-in-python/
'''

In [ ]:
with open('purpose_classification_new/entire_training/all_dataset_complete_data.json', 'r') as f:
    f_data = json.load(f)

with open('purpose_classification_new/entire_training/entire_dataset_inference_results.json', 'r') as f:
    results = json.load(f)

idx_set = set()
for k, item in results.items():
    if int(item) == 0:
        idx_set.add(int(k))

print(len(idx_set))

# text2country
text2country = {}
for data in f_data:
    index = data[0]
    if int(index) in idx_set:
        if data[2] in text2country:
            text2country[data[2]].append(data[3]['country_name'])
        else:
            text2country[data[2]] = [data[3]['country_name']]

print(len(text2country))

    

14751
11139


In [ ]:
# make entire data
list(text2country.keys())
new_text2country = {}
country_text = []
for k, item in text2country.items():
    country_text.append(k.lower())
    new_text2country[k.lower()] = item
country_text[:10]

['this is a nice charger but you can tell it was made cheaply in china.  when it is charging the phone, the car radio gets lots of static.  not so much that i have to stop charging but like when you are near power lines and the radio station is far away.so, no rf shielding.i gave it 4 stars because it works fine for me, but if you listen to the radio, you might consider it is more like 2 or 3 stars.',
 "it was because i could no longer obtain good quality bl-5c's that i finally gave up my nokia phone.i think the last battery i tried in it was one of these, it actually had worse performance than the battery i had used for a year and was trying to replace.these might be phonies from china or just so old that they are chemically inert, but i am afraid i can't recommend these.",
 'this is the 3rd "oem" battery i bought via amazon marketplace -- as with the others, it was terrible. i ordered form the oem shop a br50. they sent a br56 which is supposed to be even better with 3 hours talk tim

## make vocab

In [ ]:
# Cleaing the text
import nltk
nltk.download('punkt')
# from nltk.corpus import stopwords
# nltk.download('stopwords')
def pre_clearn(processed_article):
    processed_article = re.sub('[^a-zA-Z]', ' ', processed_article)
    processed_article = re.sub(r'\s+', ' ', processed_article)
    return processed_article

processed_article = [pre_clearn(text) for text in country_text]
print(processed_article[:10])

# all_article = ' '.join(processed_article)
# Preparing the dataset
# all_sentences = [nltk.sent_tokenize(sent) for sent in processed_article]
# print(all_sentences[:10])

all_words = [nltk.word_tokenize(sent) for sent in processed_article]

# Removing Stop Words
from nltk.corpus import stopwords
nltk.download('stopwords')

final_corpus = [[w for w in ws if w not in stopwords.words('english')] for ws in all_words]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['this is a nice charger but you can tell it was made cheaply in china when it is charging the phone the car radio gets lots of static not so much that i have to stop charging but like when you are near power lines and the radio station is far away so no rf shielding i gave it stars because it works fine for me but if you listen to the radio you might consider it is more like or stars ', 'it was because i could no longer obtain good quality bl c s that i finally gave up my nokia phone i think the last battery i tried in it was one of these it actually had worse performance than the battery i had used for a year and was trying to replace these might be phonies from china or just so old that they are chemically inert but i am afraid i can t recommend these ', 'this is the rd oem battery i bought via amazon marketplace as with the others it was terrible i ordered form the oem sho

In [ ]:
# make the dictionary
from gensim.models import Word2Vec

word2vec_1 = Word2Vec(final_corpus, min_count=10)
vocabulary = word2vec_1.wv.vocab
print(vocabulary)

{'nice': <gensim.models.keyedvectors.Vocab object at 0x7facfafea850>, 'charger': <gensim.models.keyedvectors.Vocab object at 0x7facfafea490>, 'tell': <gensim.models.keyedvectors.Vocab object at 0x7facfafea410>, 'made': <gensim.models.keyedvectors.Vocab object at 0x7facfafea450>, 'cheaply': <gensim.models.keyedvectors.Vocab object at 0x7facfafeafd0>, 'china': <gensim.models.keyedvectors.Vocab object at 0x7facfafed5d0>, 'charging': <gensim.models.keyedvectors.Vocab object at 0x7facfafed750>, 'phone': <gensim.models.keyedvectors.Vocab object at 0x7facfafed950>, 'car': <gensim.models.keyedvectors.Vocab object at 0x7facfafeaa90>, 'radio': <gensim.models.keyedvectors.Vocab object at 0x7facfafed6d0>, 'gets': <gensim.models.keyedvectors.Vocab object at 0x7facfafed990>, 'lots': <gensim.models.keyedvectors.Vocab object at 0x7facfafed9d0>, 'static': <gensim.models.keyedvectors.Vocab object at 0x7facfafeda10>, 'much': <gensim.models.keyedvectors.Vocab object at 0x7facfafeda90>, 'stop': <gensim.mod

## counting


In [ ]:
done = ["reviews_Cell_Phones_and_Accessories_5", "reviews_Toys_and_Games_5", 
        "reviews_Office_Products_5", "reviews_Pet_Supplies_5", 'reviews_Tools_and_Home_Improvement_5',
        "reviews_Automotive_5", "reviews_Digital_Music_5", "reviews_Musical_Instruments_5", 
        "reviews_Clothing_Shoes_and_Jewelry_5", "reviews_Amazon_Instant_Video_5"]

country_worth2lookat = {'china':['china', 'ch'], 'us':['usa', 'us', 'america', 'united states', 'the states'], 
                      'uk':['uk', 'england', 'britain', 'ireland', 'scotland'], 'japan':['japan'], 'mexico':['mexico'], 'germany': ['germany'], 
                      'canada':['canada'], 'india':['india'], 'australia':['australia'], 'taiwan':['taiwan'], 'vietnam':['vietnam'], 
                      'thailand':['thailand'], 'korea':['korea'], 'iraq':['iraq'], 'russia':['russia']} #15


nationality_worth2lookat = {'american': ['american'], 'english': ['english'], 'german':['german'], 
                          'chinese':['chinese'], 'british':['british', 'scottish', 'irish'], 
                          'spanish':['spanish'], 'french':['french'], 'japanese':['japanese'], 
                          'european':['european'], 'australian':['australian'], 
                          'african':['african'], 'asian':['asian'], 'italian':['italian'], 
                          'canadian':['canadian'],  'indian':['indian'], 
                          'russian':['russian'], 'mexican':['mexican']} # 17

# build word2label
word2label_country = {}
word2label_nationality = {}
for key, item in country_worth2lookat.items():
    for word in item:
        word2label_country[word] = key
for key, item in nationality_worth2lookat.items():
    for word in item:
        word2label_nationality[word] = key

In [ ]:
# 1-gram counting for countries
vocab = [item for item in word2vec_1.wv.vocab]
vocab_set = set(vocab)

def pre_clearn(processed_article):
    processed_article = re.sub('[^a-zA-Z]', ' ', processed_article)
    processed_article = re.sub(r'\s+', ' ', processed_article)
    all_sentences = nltk.sent_tokenize(processed_article)
    all_words = [nltk.word_tokenize(sent) for sent in all_sentences]
    review = []
    for words in all_words:
        review = review + words
    return review

def get_counts(review):
    count_dict = {v:0 for v in vocab}
    review = pre_clearn(review)
    for w in review:
        if w in count_dict:
            count_dict[w] += 1
    
    return count_dict

# new_text2country
# country_text
# calculate the counts for each sentence
# print(country)
country_count_dict = {country:{v:0 for v in vocab} for country in list(country_worth2lookat.keys())}
lookup_t = {}
lookup_t = {idx:new_text2country[sentence] for idx, sentence in enumerate(country_text)}  # idx -> country
review_var = [get_counts(t) for t in country_text] # all sentence -> variable
print(len(review_var))
country2idx = {country: [idx for idx, c_list in lookup_t.items() if country in c_list] for country in list(country_worth2lookat.keys())} # country -> list of idxes

for c, idx_list in country2idx.items():
    print(c)
    for idx in idx_list:
        for v, counts in review_var[idx].items():
            country_count_dict[c][v] += counts

    sorted_words = sorted([(key, f) for key, f in country_count_dict[c].items()], key=lambda k:k[1])[::-1]
    print(sorted_words)
    with open('/'.join(['bagofwords_new', c+'_bow.json']), 'w') as f:
        json.dump(sorted_words, f)


In [ ]:
# calculate the number of words for each country:
country_count_dict = {country:0 for country in list(country_worth2lookat.keys())}

def find_length(review):
    count_dict = {v:0 for v in vocab}
    review = pre_clearn(review)
    return len(review)

country2text = {country:[] for country in list(country_worth2lookat.keys())}
for country_list in list(word2label_country.keys()): # ['usa']:'usa', 'us':'usa
    for text, cs in new_text2country.items(): # text, list of contry
        saved_list = []
        cs = [c.lower() for c in cs]
        if country_list.lower() in cs and word2label_country[country_list] not in saved_list:
            country2text[word2label_country[country_list]].append(text)
            saved_list.append(word2label_country[country_list])
    
for country in country_worth2lookat.keys():
    review_var = list(map(lambda t: find_length(t), country2text[country]))
    length_sum = sum(review_var)
    country_count_dict[country] = length_sum

print(country_count_dict)

In [ ]:
with open('/'.join(['bagofwords_new', 'country2text.json']), 'w') as f:
    json.dump(country2text, f)

with open('/'.join(['bagofwords_new', 'country_count_dict.json']), 'w') as f:
    json.dump(country_count_dict, f)

## IDF

In [ ]:
# calculate idf
import numpy as np

def generate_binary_bow(text):
    var = []
    for v in vocab:
        if v in text:
            var.append(1)
        else:
            var.append(0)
    return var

# with open('/'.join(['bagofwords', 'all_country_text.json']), 'r') as f:
#     text = json.load(f)

# n_total_document = len(text)
# print(n_total_document)
# print(vocab)

pre_cleaned_worded_review = [pre_clearn(t) for t in country_text]
variables = np.array([generate_binary_bow(t) for t in pre_cleaned_worded_review])
print(variables.shape)
IDF = np.log(len(country_text)/np.sum(variables, axis=0))
IDF_dict = {v:i for v,i in zip(vocab, IDF)}
with open('/'.join(['bagofwords_new', 'country_vocab_idf.json']), 'w') as f:
    json.dump(IDF_dict, f)
print(IDF_dict)
print(sorted([(k, w)for k, w in IDF_dict.items()], key=lambda x:x[1]))



## TF-IDF

In [ ]:
review_var = [get_counts(t) for t in country_text]
length = [find_length(t) for t in country_text]

In [ ]:
# calculate the TF-IDF
# first calculate the TF-IDF for each review
# combine them by lookup_t idx->country
# IDF_dict


print(review_var[:3])
print(length[:3])

tfidf = {country:[] for country in list(country_worth2lookat.keys())} #{'country': []...}
for idx, var_l in enumerate(zip(review_var, length)):
    saved_c = []
    var, leng = var_l
    for k in var.keys():
        var[k] = var[k]/leng*IDF_dict[k]
    
    for c in lookup_t[idx]:
        if c.lower() in word2label_country and word2label_country[c.lower()] not in saved_c:
            tfidf[word2label_country[c.lower()]].append(var)
            saved_c.append(word2label_country[c.lower()])



[{'nice': 1.3367402183114617e-05, 'charger': 9.563232833474654e-05, 'tell': 5.6612153959122855e-05, 'made': 7.504125384932018e-08, 'cheaply': 0.00016775300736180233, 'china': 4.949522910381726e-07, 'charging': 0.00026097743267620136, 'phone': 4.7386265543402056e-05, 'car': 9.090262580713725e-05, 'radio': 0.0004924792801256191, 'gets': 6.573612318959737e-05, 'lots': 0.00011357715431066342, 'static': 0.0004583739222350915, 'much': 7.655364553942398e-06, 'stop': 0.00010666601668846192, 'like': 3.230575079521235e-06, 'near': 0.00011398720741828265, 'power': 4.8931884310844904e-05, 'lines': 0.0001785843143833061, 'station': 0.0003961511623902852, 'far': 2.8500890878850713e-05, 'away': 4.200739012937361e-05, 'gave': 5.9930985417985604e-05, 'stars': 7.67367814368526e-05, 'works': 3.0391007804754897e-05, 'fine': 3.379955282246705e-05, 'listen': 0.00014912320844625893, 'might': 3.830909295358916e-05, 'consider': 0.00011236441530198632, 'could': 0.0, 'longer': 0.0, 'obtain': 0.0, 'good': 0.0, 'q

KeyError: ignored

In [ ]:
country_tf_idf = {}
for k, item in tfidf.items(): #{'country': []...}
    if k == 'iraq':
        continue
    print(k, len(item))
    num_sent = len(item)
    temp = {v:0 for v in vocab}
    for sent_var in item: #[{'made':0.98}]
        for k1, v in sent_var.items(): # sum
            temp[k1] += v
    for k2 in sent_var.keys(): # devide
        temp[k2] = temp[k]/num_sent
    
    country_tf_idf[k] = temp
    
    sorted_words = sorted([(key, f) for key, f in country_tf_idf[k].items()], key=lambda x:x[1])[::-1]
    print([k_ for k_, _ in sorted_words][:100])

with open('/'.join(['bagofwords_new', 'review_level_country_tf_idf.json']), 'w') as f:
    json.dump(country_tf_idf, f)

In [ ]:
with open('/'.join(['bagofwords_new', 'review_level_country_tf_idf.json']), 'r') as f:
    tfidf = json.load(f)

## evaluation

In [ ]:
import nltk
from nltk.corpus import opinion_lexicon
nltk.download('opinion_lexicon')
pos_list=set(opinion_lexicon.positive())
neg_list=set(opinion_lexicon.negative())

[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Unzipping corpora/opinion_lexicon.zip.


In [ ]:
with open('/'.join(['bagofwords_new', 'review_level_country_tf_idf.json']), 'r') as f:
    country_tf_idf = json.load(f)

products_set = set(['battery', 'dog', 'food', 'plastic', 'toy', 'toys', 'album', 'phone',  'pencils' ,
            'song', 'music', 'cat', 'tool', 'cats', 'game', 'puzzle', 'chicken','screen', 'game',
            'iphone', 'cards', 'charger', 'jeans', 'movie', 'guitar', 'map', 'wrench', 'fish', 'canned', 'broth'
            ])
aspects_set = set(['quality', 'price', 'size', 'shipping', 'color', 'service', 'material'])
country_set = set(list(country_worth2lookat.keys()))
sentiment_set = set(list(pos_list)+list(neg_list))
for country, w_s in  country_tf_idf.items():
    print(country)
    w_s = sorted([(key, f) for key, f in w_s.items()], key=lambda k:k[1])[::-1]
    w_s = w_s[:200]
    country_product = []
    p_aspects = []
    country_matching = []
    sentiment_words = []
    for w, s in w_s:
        # product 
        if w in products_set:
            country_product.append(w)
        if w in aspects_set:
            p_aspects.append(w)
        if w in country_set:
            country_matching.append(w)
        if w in sentiment_set:
            sentiment_words.append(w)
    
    print(country_product,
            p_aspects,
            country_matching,
            sentiment_words)



china
['charger'] [] ['china'] ['cheaply', 'nice', 'fidelity', 'depressing', 'madman', 'inspirational', 'glorious', 'bombastic', 'poignant']
us
[] ['material'] ['us'] ['failure', 'bother', 'simpler', 'protect', 'effective', 'cheapest', 'elegant', 'ready', 'clever', 'satisfied', 'thicker', 'wow', 'crap', 'dust', 'impossible', 'smoother', 'struggle', 'attractive', 'distracting', 'brighter', 'dynamic', 'crooked', 'lighter', 'correctly', 'questionable', 'benefits', 'disagree', 'spite', 'awesome', 'hate', 'convenient', 'saver', 'doubt', 'blow', 'pros', 'dump', 'unfortunately', 'enough', 'powerful', 'compact', 'sweaty', 'ridiculously', 'cheaper', 'avid']
uk
[] [] ['uk', 'taiwan'] ['win', 'successful', 'beauty', 'funny', 'smell', 'grace', 'versatile', 'ease', 'unable', 'garbage', 'grateful', 'dying', 'superior', 'subdued', 'winner', 'sturdier', 'killed', 'flake', 'glitter', 'smooth', 'dumping', 'fat', 'classy', 'criticism', 'retract', 'flaw', 'noise', 'sucks', 'cute', 'ridiculous', 'breaking'

# For country, country level TF-IDF

---
This is used in the final submission


In [ ]:
done = ["reviews_Cell_Phones_and_Accessories_5", "reviews_Toys_and_Games_5", 
        "reviews_Office_Products_5", "reviews_Pet_Supplies_5", 'reviews_Tools_and_Home_Improvement_5',
        "reviews_Automotive_5", "reviews_Digital_Music_5", "reviews_Musical_Instruments_5", 
        "reviews_Clothing_Shoes_and_Jewelry_5", "reviews_Amazon_Instant_Video_5"]

country_worth2lookat = {'china':['china', 'ch'], 'us':['usa', 'us', 'america', 'united states', 'the states'], 
                      'uk':['uk', 'england', 'britain', 'ireland', 'scotland'], 'japan':['japan'], 'mexico':['mexico'], 'germany': ['germany'], 
                      'canada':['canada'], 'india':['india'], 'australia':['australia'], 'taiwan':['taiwan'], 'vietnam':['vietnam'], 
                      'thailand':['thailand'], 'korea':['korea'], 'iraq':['iraq'], 'russia':['russia']} #15


nationality_worth2lookat = {'american': ['american'], 'english': ['english'], 'german':['german'], 
                          'chinese':['chinese'], 'british':['british', 'scottish', 'irish'], 
                          'spanish':['spanish'], 'french':['french'], 'japanese':['japanese'], 
                          'european':['european'], 'australian':['australian'], 
                          'african':['african'], 'asian':['asian'], 'italian':['italian'], 
                          'canadian':['canadian'],  'indian':['indian'], 
                          'russian':['russian'], 'mexican':['mexican']} # 17

# build word2label
word2label_country = {}
word2label_nationality = {}
for key, item in country_worth2lookat.items():
    for word in item:
        word2label_country[word] = key
for key, item in nationality_worth2lookat.items():
    for word in item:
        word2label_nationality[word] = key

In [ ]:
text2country # text -> ['china', 'usa' ...]

# first of all convert ['china', 'us'] -> ['china', 'usa']

for text in text2country.keys():
    country = text2country[text]
    c_set = set()
    for c in country:
        if c in word2label_country:
            c_set.add(word2label_country[c])
    text2country[text] = list(c_set)


In [ ]:
# text2country -> common country -> text
common_country2text = {}
for key, country in text2country.items():
    for c in country:
        if c in common_country2text:
            common_country2text[c].append(key)
        else:
            common_country2text[c] = [key]


In [ ]:
# 1-gram counting for countries
vocab = [item for item in word2vec_1.wv.vocab]
vocab_set = set(vocab)

def pre_clearn(processed_article):
    processed_article = re.sub('[^a-zA-Z]', ' ', processed_article)
    processed_article = re.sub(r'\s+', ' ', processed_article)
    all_sentences = nltk.sent_tokenize(processed_article)
    all_words = [nltk.word_tokenize(sent) for sent in all_sentences]
    review = []
    for words in all_words:
        review = review + words
    return review

def get_counts(review):
    count_dict = {v:0 for v in vocab}
    review = pre_clearn(review)
    for w in review:
        if w in count_dict:
            count_dict[w] += 1
    
    return count_dict


for country in list(country_worth2lookat.keys()):
    if country == 'iraq':
        continue

    print(country)
    country_count_dict = {v:0 for v in vocab}
    text = common_country2text[country]
    review_var = list(map(lambda t: get_counts(t.lower()), text))
    for v in vocab:
        for var in review_var:
            country_count_dict[v] += var[v]

        
    sorted_words = sorted([(key, f) for key, f in country_count_dict.items()], key=lambda k:k[1])[::-1]
    print(sorted_words)
    with open('/'.join(['bagofwords_new', country+'_country_level_bow.json']), 'w') as f:
        json.dump(sorted_words, f)


In [ ]:
# calculate the number of words for each country:
country_count_dict = {}

def find_length(review):
    count_dict = {v:0 for v in vocab}
    review = pre_clearn(review)
    return len(review)

for country in list(country_worth2lookat.keys()):
    if country == 'iraq':
        continue
    # print(country)
    length_sum = 0
    # for name in done:
    #     path2file = '/'.join(['bagofwords', name, country+'_country_text.json'])
    #     with open(path2file, 'r') as f:
    #         text = json.load(f)
    text = common_country2text[country]
    review_var = list(map(lambda t: find_length(t.lower()), text))
    length_sum = sum(review_var)
    country_count_dict[country] = length_sum

print(country_count_dict)

In [ ]:
# calculate idf
import numpy as np

def generate_binary_bow(text):
    var = []
    for v in vocab:
        if v in text:
            var.append(1)
        else:
            var.append(0)
    
    return var

country_text = []
for k, item in text2country.items():
    country_text.append(k.lower())
    # new_text2country[k.lower()] = item
text = country_text

n_total_document = len(text)
print(n_total_document)
print(vocab)

pre_cleaned_worded_review = [pre_clearn(t.lower()) for t in text]
variables = np.array([generate_binary_bow(t) for t in pre_cleaned_worded_review])
print(variables.shape)
IDF = np.log(n_total_document/np.sum(variables, axis=0))
IDF_dict = {v:i for v,i in zip(vocab, IDF)}
with open('/'.join(['bagofwords_new', 'country_country_level_vocab_idf.json']), 'w') as f:
    json.dump(IDF_dict, f)
print(IDF_dict)
print(sorted([(k, w)for k, w in IDF_dict.items()], key=lambda x:x[1])[::-1])



In [ ]:
# calculate the TF-IDF

tfidf = {}
for country in list(country_worth2lookat.keys()):
    if country == 'iraq':
        continue
    with open('/'.join(['bagofwords_new', country+'_country_level_bow.json']), 'r') as f:
        sorted_words = json.load(f)
    print(country)
    tfidf[country] = {}
    for w, c in sorted_words:
        tfidf[country][w] = int(c)/float(country_count_dict[country])*IDF_dict[w]

    sorted_words_new = sorted([(key, f) for key, f in tfidf[country].items()], key=lambda k:k[1])[::-1]

    print([k for k,v in sorted_words_new][:200])

with open('/'.join(['bagofwords_new', 'country_country_level_tf_idf.json']), 'w') as f:
    json.dump(tfidf, f)


In [ ]:
with open('/'.join(['bagofwords_new', 'country_country_level_tf_idf.json']), 'r') as f:
    country_tf_idf = json.load(f)

products_set = set(['battery', 'dog', 'food', 'plastic', 'toy', 'toys', 'album', 'phone',  'pencils' , 'movie', 'watches',
            'song', 'music', 'cat', 'tool', 'cats', 'game', 'puzzle', 'chicken','screen', 'game', 'piano', 'guitar'
            'iphone', 'cards', 'charger', 'jeans', 'movie', 'guitar', 'map', 'wrench', 'fish', 'canned', 'broth'
            ])
aspects_set = set(['quality', 'price', 'size', 'shipping', 'color', 'service', 'material', 'taste'])
country_set = set(list(country_worth2lookat.keys()))
sentiment_set = set(list(pos_list)+list(neg_list))
for country, w_s in  country_tf_idf.items():
    print(country)
    w_s = sorted([(key, f) for key, f in w_s.items()], key=lambda k:k[1])[::-1]
    w_s = w_s[:200]
    country_product = []
    p_aspects = []
    country_matching = []
    sentiment_words = []
    for w, s in w_s:
        # product 
        if w in products_set:
            country_product.append((w))
        if w in aspects_set:
            p_aspects.append((w))
        if w in country_set:
            country_matching.append((w))
        if w in sentiment_set:
            sentiment_words.append((w))
    
    print(country_product,
            p_aspects,
            country_matching,
            sentiment_words)



china
['phone', 'battery', 'dog', 'food', 'plastic', 'charger', 'tool', 'toy', 'chicken', 'toys'] ['quality', 'price', 'size', 'shipping', 'color'] ['china', 'us'] ['like', 'good', 'well', 'cheap', 'great', 'better', 'nice', 'work', 'enough', 'pretty', 'love', 'easy', 'right', 'led', 'problem', 'works', 'bad', 'worth', 'top', 'fine', 'jerky', 'hard', 'recommend', 'best', 'bright', 'perfect']
us
['dog', 'food', 'album', 'chicken', 'toys', 'toy', 'tool', 'song', 'cat', 'cats', 'plastic', 'phone', 'battery', 'music'] ['quality', 'price', 'size'] ['us', 'china'] ['like', 'good', 'love', 'great', 'well', 'better', 'best', 'work', 'nice', 'loves', 'easy', 'right', 'enough', 'jerky', 'hard', 'free', 'top', 'perfect', 'recommend', 'happy', 'excellent', 'knife', 'problem', 'expensive', 'pretty', 'works', 'comfortable', 'fine', 'smell', 'safe']
uk
['album', 'song', 'music', 'guitar', 'piano', 'movie', 'phone'] ['material'] ['uk', 'us'] ['best', 'great', 'love', 'like', 'good', 'well', 'classic',

In [ ]:
common_country2text['vietnam']